In [1]:
######################################################## LIBRARIES ########################################################
import tensorflow as tf

import numpy as np

from sklearn.metrics import classification_report

import CONSTANTS as c
######################################################## CONSTANTS ########################################################
ATTENTION_KEY_DIMS =c.ATTENTION_KEY_DIMS
ATTENTION_NR_OF_HEADS = c.ATTENTION_NR_OF_HEADS
ENCODER_DENSE_DIMS = c.ENCODER_DENSE_DIMS
DROPOUT_RATE = c.DROPOUT_RATE
NR_OF_ENCODER_BLOCKS = c.NR_OF_ENCODER_BLOCKS

CHECKPOINT_PATH =  c.CHECKPOINT_PATH
DISERT_DATA_PATH = c.DISERT_DATA_PATH

STOPPER_THRESHOLD = c.STOPPER_THRESHOLD
######################################################## DATA SOURCE ########################################################
X_MLM = np.load(f'{DISERT_DATA_PATH}\X_MLM.npy')
Y_MLM = np.load(f'{DISERT_DATA_PATH}\Y_MLM.npy')
X_NSP = np.load(f'{DISERT_DATA_PATH}\X_NSP.npy')
Y_NSP = np.load(f'{DISERT_DATA_PATH}\Y_NSP.npy')

In [2]:
# class PositionEncoder(tf.Module):
#     def __init__(self, input_dim, output_dim ):
#         super(PositionEncoder, self).__init__()
        
#         self.input_dim = input_dim
#         self.output_dim = output_dim
        
#         self.pos_enc = self.get_pos_encoding_matrix(input_dim, output_dim)
        
        
#     def __call__(self, x):
#         y = tf.keras.layers.Embedding(
#             input_dim= self.input_dim,
#             output_dim=self.output_dim,
#             weights=[self.pos_enc],
#             name="position_embedding",
#         )(x)
        
#         return y

        
#     def get_pos_encoding_matrix(self, max_len,d_emb ):
#         pos_enc = np.array(
#             [
#                 [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
#                 if pos != 0
#                 else np.zeros(d_emb)
#                 for pos in range(max_len)
#             ]
#         )
#         pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
#         pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
#         return pos_enc



# class TransformerEncoder(tf.Module):
#     def __init__(self, key_dim, num_heads, ff_dim, dropout):
#         super(TransformerEncoder, self).__init__()
#         self.key_dim = key_dim
#         self.num_heads = num_heads
#         self.ff_dim = ff_dim
#         self.dropout = dropout
        
        
#     def __call__(self, x):
#         # Normalization and Attention
#         y = tf.keras.layers.LayerNormalization(epsilon =1e-6)(x)
        
#         y = tf.keras.layers.MultiHeadAttention(
#             key_dim=self.key_dim,
#             num_heads = self.num_heads,
#             dropout = self.dropout
#         )(y,y) # self attention
        
        
#         y = tf.keras.layers.Dropout(self.dropout)(y)
#         res = y + x
        
#         # Feed Forward Part
#         y = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(res)
#         y = tf.keras.layers.Conv1D(filters = self.ff_dim, kernel_size=  1, activation = 'relu')(y)
#         y = tf.keras.layers.Dropout(self.dropout)(y)
#         y = tf.keras.layers.Conv1D(filters=x.shape[-1], kernel_size = 1)(y)
        
#         return y + res
        

# mlm_input = tf.keras.layers.Input((X_MLM.shape[1], X_MLM.shape[2]), name = 'mlm_input')
# nsp_input = tf.keras.layers.Input((X_NSP.shape[1], X_NSP.shape[2]), name = 'nsp_input')

# x = tf.keras.layers.Concatenate(axis = 1)([mlm_input, nsp_input])

# x_positions = (tf.range(start=0, limit=x.shape[1], delta=1))
# x_position_embeddings = PositionEncoder(
#     input_dim = x.shape[1] , 
#     output_dim = x.shape[2]
# )(x_positions)
# x = tf.add(x, x_position_embeddings)

# for i in range(NR_OF_ENCODER_BLOCKS):
#     x = TransformerEncoder(ATTENTION_KEY_DIMS, ATTENTION_NR_OF_HEADS, ENCODER_DENSE_DIMS, DROPOUT_RATE)(x)

# mlm_output = tf.keras.layers.Dense(X_MLM.shape[2], activation = 'softmax', name = 'mlm_classifier')(x)

# x = tf.keras.layers.Flatten()(x)
# nsp_output = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'nsp_classifier')(x)

# DisERT_model = tf.keras.Model(inputs = [mlm_input, nsp_input], outputs= [mlm_output, nsp_output])

def mlm_custom_loss(y_true, y_pred):
    # y_true for mlm is without cls token. So we should ignore the cls_token time step of y_pred.
    # since cls token is the latest latest token of prediction, we ignore that token.
    y_pred_without_special_tokens = y_pred[:, :-1, :]
    non_masks = tf.not_equal(y_true, tf.cast(tf.ones(y_true.shape[2])*-1, tf.dtypes.float32))[:,:,0]
    
    y_pred_non_mask = tf.boolean_mask(y_pred_without_special_tokens,non_masks)
    y_true_non_mask = tf.boolean_mask(y_true,non_masks)
    
    oBinCE = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    loss = oBinCE(y_true_non_mask , y_pred_non_mask)
    loss = tf.reduce_mean(loss)
    
    return loss

def nsp__custom_loss(y_true, y_pred):
    oBinCE = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    loss = oBinCE(y_true , y_pred)
    loss = tf.reduce_mean(loss)
    
    return  loss
        
# DisERT_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
#     loss = [mlm_custom_loss, nsp__custom_loss]
# )

# print(DisERT_model.summary())
# tf.keras.utils.plot_model(DisERT_model, show_shapes=True)

In [3]:
X_MLM = X_MLM[:4000]
X_NSP = X_NSP[:4000]
Y_MLM = Y_MLM[:4000]
Y_NSP = Y_NSP[:4000]

In [20]:
import time
import os

class CsvLogger(tf.keras.callbacks.Callback):
    
    def __init__(self, sFilePath, sDelimeter = ';'):
        sFilePath = f'{sFilePath}.txt'
        self.sFilePath = sFilePath
        self.sDelimeter = sDelimeter


    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
        

    def on_epoch_end(self, epoch, logs={}):
        endTime = time.time()
        
        logs['start'] = self.epoch_time_start
        logs['end'] = endTime
        logs['duration'] = endTime - self.epoch_time_start
        
        logs['epoch'] = epoch
        logs['learning_rate'] = self.model.optimizer.lr.numpy()
        
        if os.path.exists(self.sFilePath) == False:
            with open(self.sFilePath, 'a') as f: 
                f.write(self.sDelimeter.join([str(i) for i in logs.keys()]))
                f.write('\n') 
                
        with open(self.sFilePath, 'a') as f: 
            f.write(self.sDelimeter.join([str(i) for i in logs.values()]))
            f.write('\n')
            
            
class ThresholdStopper(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        self.threshold = threshold


    def on_batch_end(self, batch, logs={}):
        if logs.get('loss') <= self.threshold:
             self.model.stop_training = True

In [5]:
sCheckPointFilePath = f'{CHECKPOINT_PATH}\model'
sCsvLogFilePath = f'{CHECKPOINT_PATH}\log'

DisERT_model = tf.keras.models.load_model(sCheckPointFilePath,
                                         custom_objects ={'mlm_custom_loss':mlm_custom_loss, 'nsp__custom_loss':nsp__custom_loss} )

In [22]:
# try:
#     DisERT_model.load_weights(sCheckPointFilePath)
# except:
#     print('There is no pre-trained model checkpoint')

oCsvLogger=  CsvLogger(sCsvLogFilePath)

oDisERTCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=sCheckPointFilePath,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True
)

oLearningRateReducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss', 
    factor=0.80,
    patience=3, 
    min_lr=1e-4
)

oEarlyStopper = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

oThresholdStopper = ThresholdStopper(STOPPER_THRESHOLD)

DisERT_model.fit(
    x = [X_MLM, X_NSP], 
    y = [Y_MLM, Y_NSP], 
    batch_size= 512,
    epochs=2,
    verbose=1,
    callbacks = [oDisERTCheckPoint, oCsvLogger, oLearningRateReducer, oEarlyStopper, oThresholdStopper]
)

Epoch 1/2
1/8 [==>...........................] - ETA: 3s - loss: 0.1648 - mlm_classifier_loss: 0.0360 - nsp_classifier_loss: 0.1289

INFO:tensorflow:Assets written to: C:\Users\yunus\Desktop\Checkpoints\DisERT\model\assets


INFO:tensorflow:Assets written to: C:\Users\yunus\Desktop\Checkpoints\DisERT\model\assets


8/8 [==============================] - 9s 1s/step - loss: 0.1648 - mlm_classifier_loss: 0.0360 - nsp_classifier_loss: 0.1289 - start: 1676676027.7823 - end: 1676676036.3023 - duration: 8.5200 - epoch: 0.0000e+00 - learning_rate: 0.0010 - lr: 0.0010
